In [1]:
%matplotlib inline
import itertools
import os
os.environ['CUDA_VISIBLE_DEVICES']=""
import numpy as np
import gpflow
import gpflow.training.monitor as mon
import numbers
import matplotlib.pyplot as plt
import tensorflow as tf

# Demo: `gpflow.training.monitor`
In this notebook we'll demo how to use `gpflow.training.monitor` for logging the optimisation of a GPflow model.

## Creating the GPflow model
We first generate some random data and create a GPflow model.

Under the hood, GPflow gives a unique name to each model which is used to name the Variables it creates in the TensorFlow graph containing a random identifier. This is useful in interactive sessions, where people may create a few models, to prevent variables with the same name conflicting. However, when loading the model, we need to make sure that the names of all the variables are exactly the same as in the checkpoint. This is why we pass name="SVGP" to the model constructor, and why we use gpflow.defer_build().

In [2]:
np.random.seed(0)
X = np.random.rand(10000, 1) * 10
Y = np.sin(X) + np.random.randn(*X.shape)
Xt = np.random.rand(10000, 1) * 10
Yt = np.sin(Xt) + np.random.randn(*Xt.shape)

with gpflow.defer_build():
    m = gpflow.models.SVGP(X, Y, gpflow.kernels.RBF(1), gpflow.likelihoods.Gaussian(),
                           Z=np.linspace(0, 10, 5)[:, None],
                           minibatch_size=100, name="SVGP")
    m.likelihood.variance = 0.01
m.compile()

Let's compute log likelihood before the optimisation

In [3]:
print('LML before the optimisation: %f' % m.compute_log_likelihood())

LML before the optimisation: -1271605.621944


We will be using a TensorFlow optimiser. All TensorFlow optimisers have a support for `global_step` variable. Its purpose is to track how many optimisation steps have occurred. It is useful to keep this in a TensorFlow variable as this allows it to be restored together with all the parameters of the model.

The code below creates this variable using a monitor's helper function. It is important to create it before building the monitor in case the monitor includes a checkpoint task. This is because the checkpoint internally uses the TensorFlow Saver which creates a list of variables to save. Therefore all variables expected to be saved by the checkpoint task should exist by the time the task is created.

In [4]:
session = m.enquire_session()
global_step = mon.create_global_step(session)

## Construct the monitor

Next we need to construct the monitor. `gpflow.training.monitor` provides classes that are building blocks for the monitor. Essengially, a monitor is a function that is provided as a callback to an optimiser. It consists of a number of tasks that may be executed at each step, subject to their running condition.

In this example, we want to:
- log certain scalar parameters in TensorBoard,
- log the full optimisation objective (log marginal likelihood bound) periodically, even though we optimise with minibatches,
- store a backup of the optimisation process periodically,
- log performance for a test set periodically.

We will define these tasks as follows:

In [5]:
print_task = mon.PrintTimingsTask().with_name('print')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

sleep_task = mon.SleepTask(0.01).with_name('sleep').with_name('sleep')

saver_task = mon.CheckpointTask('./monitor-saves').with_name('saver')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

file_writer = mon.LogdirWriter('./model-tensorboard')

model_tboard_task = mon.ModelToTensorBoardTask(file_writer, m).with_name('model_tboard')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

lml_tboard_task = mon.LmlToTensorBoardTask(file_writer, m).with_name('lml_tboard')\
    .with_condition(mon.PeriodicIterationCondition(100))\
    .with_exit_condition(True)

As the above code shows, each task can be assigned a name and running conditions. The name will be shown in the task timing summary.

There are two different types of running conditions: `with_condition` controls execution of the task at each iteration in the optimisation loop. `with_exit_condition` is a simple boolean flag indicating that the task should also run at the end of optimisation.
In this example we want to run our tasks periodically, at every iteration or every 10th or 100th iteration.

Notice that the two TensorBoard tasks will write events into the same file. It is possible to share a file writer between multiple tasks. However it is not possible to share the same event location between multiple file writers. An attempt to open two writers with the same location will result in error.


## Custom tasks
We may also want to perfom certain tasks that do not have pre-defined `Task` classes. For example, we may want to compute the performance on a test set. Here we create such a class by extending `BaseTensorBoardTask` to log the testing benchmarks in addition to all the scalar parameters.

In [6]:
class CustomTensorBoardTask(mon.BaseTensorBoardTask):
    def __init__(self, file_writer, model, Xt, Yt):
        super().__init__(file_writer, model)
        self.Xt = Xt
        self.Yt = Yt
        self._full_test_err = tf.placeholder(gpflow.settings.tf_float, shape=())
        self._full_test_nlpp = tf.placeholder(gpflow.settings.tf_float, shape=())
        self._summary = tf.summary.merge([tf.summary.scalar("test_rmse", self._full_test_err),
                                         tf.summary.scalar("test_nlpp", self._full_test_nlpp)])
    
    def run(self, context: mon.MonitorContext, *args, **kwargs) -> None:
        minibatch_size = 100
        preds = np.vstack([self.model.predict_y(Xt[mb * minibatch_size:(mb + 1) * minibatch_size, :])[0]
                            for mb in range(-(-len(Xt) // minibatch_size))])
        test_err = np.mean((Yt - preds) ** 2.0)**0.5
        self._eval_summary(context, {self._full_test_err: test_err, self._full_test_nlpp: 0.0})

        
custom_tboard_task = CustomTensorBoardTask(file_writer, m, Xt, Yt).with_name('custom_tboard')\
    .with_condition(mon.PeriodicIterationCondition(100))\
    .with_exit_condition(True)

Now we can put all these tasks into a monitor.

In [7]:
monitor_tasks = [print_task, model_tboard_task, lml_tboard_task, custom_tboard_task, saver_task, sleep_task]
monitor = mon.Monitor(monitor_tasks, session, global_step)

## Running the optimisation
We finally get to running the optimisation.

We may want to continue a previously run optimisation by resotring the TensorFlow graph from the latest checkpoint. Otherwise skip this step.

In [8]:
if os.path.isdir('./monitor-saves'):
    mon.restore_session(session, './monitor-saves')

INFO:tensorflow:Restoring parameters from ./monitor-saves/cp-1450


INFO:tensorflow:Restoring parameters from ./monitor-saves/cp-1450


In [9]:
optimiser = gpflow.train.AdamOptimizer(0.01)

with mon.Monitor(monitor_tasks, session, global_step, print_summary=True) as monitor:
    optimiser.minimize(m, step_callback=monitor, maxiter=450, global_step=global_step)

file_writer.close()

Iteration 10	total itr.rate 12.91/s	recent itr.rate 12.91/s	opt.step 1460	total opt.rate 14.63/s	recent opt.rate 14.63/s
Iteration 20	total itr.rate 19.65/s	recent itr.rate 41.10/s	opt.step 1470	total opt.rate 28.87/s	recent opt.rate 1097.44/s
Iteration 30	total itr.rate 25.08/s	recent itr.rate 56.14/s	opt.step 1480	total opt.rate 42.40/s	recent opt.rate 678.17/s
Iteration 40	total itr.rate 27.05/s	recent itr.rate 35.34/s	opt.step 1490	total opt.rate 54.93/s	recent opt.rate 486.15/s
Iteration 50	total itr.rate 29.49/s	recent itr.rate 46.20/s	opt.step 1500	total opt.rate 67.04/s	recent opt.rate 566.47/s
Iteration 60	total itr.rate 31.67/s	recent itr.rate 50.18/s	opt.step 1510	total opt.rate 78.57/s	recent opt.rate 559.69/s
Iteration 70	total itr.rate 33.26/s	recent itr.rate 47.68/s	opt.step 1520	total opt.rate 89.64/s	recent opt.rate 581.50/s
Iteration 80	total itr.rate 34.48/s	recent itr.rate 46.29/s	opt.step 1530	total opt.rate 100.18/s	recent opt.rate 563.99/s
Iteration 90	total itr.

 29%|██▉       | 29/100 [00:00<00:00, 287.63it/s]

Iteration 100	total itr.rate 36.34/s	recent itr.rate 44.46/s	opt.step 1550	total opt.rate 118.50/s	recent opt.rate 372.35/s


100%|██████████| 100/100 [00:00<00:00, 390.74it/s]


Iteration 110	total itr.rate 31.04/s	recent itr.rate 12.62/s	opt.step 1560	total opt.rate 128.22/s	recent opt.rate 712.40/s
Iteration 120	total itr.rate 31.84/s	recent itr.rate 44.55/s	opt.step 1570	total opt.rate 136.33/s	recent opt.rate 448.66/s
Iteration 130	total itr.rate 32.67/s	recent itr.rate 47.53/s	opt.step 1580	total opt.rate 145.42/s	recent opt.rate 727.10/s
Iteration 140	total itr.rate 33.40/s	recent itr.rate 47.03/s	opt.step 1590	total opt.rate 154.09/s	recent opt.rate 685.03/s
Iteration 150	total itr.rate 34.09/s	recent itr.rate 47.91/s	opt.step 1600	total opt.rate 162.80/s	recent opt.rate 779.37/s
Iteration 160	total itr.rate 34.69/s	recent itr.rate 47.25/s	opt.step 1610	total opt.rate 171.12/s	recent opt.rate 733.41/s
Iteration 170	total itr.rate 35.23/s	recent itr.rate 46.96/s	opt.step 1620	total opt.rate 179.37/s	recent opt.rate 782.62/s
Iteration 180	total itr.rate 35.76/s	recent itr.rate 47.96/s	opt.step 1630	total opt.rate 187.10/s	recent opt.rate 699.88/s
Iteratio

...

Iteration 410	total itr.rate 36.31/s	recent itr.rate 13.97/s	opt.step 1860	total opt.rate 302.36/s	recent opt.rate 657.96/s
Iteration 420	total itr.rate 36.50/s	recent itr.rate 46.19/s	opt.step 1870	total opt.rate 306.22/s	recent opt.rate 642.23/s
Iteration 430	total itr.rate 36.72/s	recent itr.rate 49.02/s	opt.step 1880	total opt.rate 310.29/s	recent opt.rate 700.88/s
Iteration 440	total itr.rate 36.90/s	recent itr.rate 46.96/s	opt.step 1890	total opt.rate 314.27/s	recent opt.rate 701.37/s
Iteration 450	total itr.rate 37.09/s	recent itr.rate 47.94/s	opt.step 1900	total opt.rate 318.08/s	recent opt.rate 683.02/s


 47%|████▋     | 47/100 [00:00<00:00, 464.14it/s]

Iteration 450	total itr.rate 36.30/s	recent itr.rate 0.00/s	opt.step 1900	total opt.rate 286.53/s	recent opt.rate 0.00/s


100%|██████████| 100/100 [00:00<00:00, 461.86it/s]


Tasks execution time summary:
print:	0.0413 (sec)
model_tboard:	0.1643 (sec)
lml_tboard:	1.3124 (sec)
custom_tboard:	1.2160 (sec)
saver:	4.0467 (sec)
sleep:	4.5388 (sec)


Now lets compute the log likelihood again. Hopefully we will see an increase in its value

In [10]:
print('LML after the optimisation: %f' % m.compute_log_likelihood())

LML after the optimisation: -13792.241274


In this example we have used the TensorFlow `AdamOptimizer`. Using `ScipyOptimizer` requires a couple of special tricks. Firstly, this optimiser works with its own copy of trained variables and updates the original ones only when the optimisation is completed. Secondly, it doesn't use the `global_step` variable. This can present a problem when doing optimisation in several stages. Monitor has to use an iteration count instead of the `global_step`, which will be reset to zero at each stage.

To adress the first problem we will provide the optimiser as one of the parameters to the monitor. The monitor will make sure the orginal variables are updated whenever we access them from a monitoring task. The second problem is addressed by creating an instance of `MonitorContext` and providing it explicitely to the `Monitor`.


In [11]:
optimiser = gpflow.train.ScipyOptimizer()
context = mon.MonitorContext()

with mon.Monitor([print_task], session, print_summary=True, optimiser=optimiser, context=context) as monitor:
    optimiser.minimize(m, step_callback=monitor, maxiter=250)

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 14991.851823
  Number of iterations: 4
  Number of functions evaluations: 11


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 14991.851823
  Number of iterations: 4
  Number of functions evaluations: 11


Iteration 4	total itr.rate 5.05/s	recent itr.rate nan/s	opt.step 4	total opt.rate 5.05/s	recent opt.rate nan/s
Tasks execution time summary:
print:	0.0414 (sec)
